# Plotting Wildfire results for MOHITO

imports

In [ ]:
import os, math
from copy import deepcopy
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter

parameters

In [ ]:
policy_eval_outputs = {
    #key is the openness level
    1: 'results/testOL1/testing_eval',
    # 2: 'results/OL1_diff_ss_stoch/OL2_15_seed_testing',
    # 3: 'results/OL1_diff_ss_stoch/OL3_15_seed_testing',
    # #negative keys are for the ablation study
    # -1: 'results/OL1_diff_ss_stoch_ablate/15_seed_testing',
    # -2: 'results/OL2_diff_ss_stoch_ablate/15_seed_testing',
    # -3: 'results/OL3_diff_ss_stoch_ablate/15_seed_testing',
}

policy_best_checkpoints = {
    key: {} for key in policy_eval_outputs.keys()
}

#rename policies from logs to as appears in legend (in plotting order)
policy_renaming = {
    'mohito': 'MOHITO',
    'mohito (Ablation)': 'MOHITO-NoTaskNodes',
    'FifoBaseline': 'FCFS',
    'WeakestBaseline': 'NTF',
    'RandomBaseline': 'Random'
}

#this is a python 3.12 thing, dicts are actually ordered
policy_order = list(policy_renaming.values())

#path to the baseline root folder
baseline_output_folder = 'baseline_test'

#colors for plotting
colors = ['#009ADE', 'red', '#AF58BA', '#FFC61E', '#F28522']

use_these_starting_states = [
    0, 1, 2
]

arrayify = lambda x: np.array(literal_eval(x))

## Preprocessing

Select the best performing checkpoint.
This can take a few minutes.

In [ ]:
for openness_level, file_path in policy_eval_outputs.items():

    dfs = [
        (pd.read_csv(os.path.join(root, file)), root.split('/')[-1])
        for root, _, files in os.walk(file_path)
        for file in files if file.endswith('.csv') and 'logits' not in file
    ]

    print(openness_level)

    dfs = pd.concat([df.assign(policy=file.split(';')[1].split('_')[0]) for df, file in dfs], ignore_index=True)

    reward_cols = [col for col in dfs.columns if 'rewards' in col]

    dfs['openness level'] = openness_level
    dfs['starting state'] = dfs['description'].apply(lambda x: int(x.split('_')[2].split(';')[1]))
    dfs['episodes'] = dfs['description'].apply(lambda x: int(x.split('_')[3].split(';')[1]))

    original_dfs = dfs.copy()

    dfs = dfs[['description', 'step', 'policy', 'openness level','starting state', 'episodes'] + reward_cols]
    dfs.drop(columns=['description'], inplace=True)

    #sum over steps
    dfs = dfs.groupby(['policy', 'openness level', 'episodes', 'starting state']).sum().reset_index()
    #mean over agent rewards
    dfs['final_rewards'] = dfs[reward_cols].mean(axis=1)
    #select specific starting states
    dfs = dfs[dfs['starting state'].isin(use_these_starting_states)]
    #mean over starting states
    dfs = dfs.groupby(['policy', 'openness level', 'episodes']).mean()

    #average over starting states / episodes generally speaking
    pivot = pd.pivot_table(dfs, index=['openness level'], columns=['policy'], values='final_rewards', aggfunc='mean')

    best_policy = pivot.idxmax(axis=1)
    pivot = pivot[best_policy]


    pivot_std = pd.pivot_table(dfs, index=['openness level'], columns=['policy'], values='final_rewards', aggfunc='std')
    pivot_std = 1.96 * pivot_std / math.sqrt(45)
    pivot_std = pivot_std[best_policy]

    policy_best_checkpoints[openness_level]['mean'] = pivot
    policy_best_checkpoints[openness_level]['ci'] = pivot_std
    policy_best_checkpoints[openness_level]['checkpoint'] = best_policy
    policy_best_checkpoints[openness_level]['dfs'] = original_dfs



Loading baselines

In [ ]:
dfs = pd.concat([
    pd.read_csv(os.path.join(root, file))
    for root, _, files in os.walk(baseline_output_folder) for file in files
    if file.endswith('.csv')
], ignore_index=True)

reward_cols = [col for col in dfs.columns if 'rewards' in col]

dfs = dfs[['description', 'step'] + reward_cols]
dfs['policy'] = dfs['description'].apply(
    lambda x: x.split('_')[0].split(';')[1])
dfs['openness level'] = dfs['description'].apply(
    lambda x: int(x.split('_')[1].split(';')[1]))
dfs['starting state'] = dfs['description'].apply(
    lambda x: int(x.split('_')[2].split(';')[1]))
dfs['episodes'] = dfs['description'].apply(
    lambda x: int(x.split('_')[3].split(';')[1]))
dfs.drop(columns=['description'], inplace=True)

#sum over steps
dfs = dfs.groupby(['policy', 'openness level', 'episodes','starting state']).sum().reset_index()
#mean over agent rewards
dfs['final_rewards'] = dfs[reward_cols].mean(axis=1)
#filter starting states 
dfs = dfs[dfs['starting state'].isin(use_these_starting_states)]
# average over starting states
dfs = dfs.groupby(['policy', 'openness level', 'episodes']).mean().reset_index()

pivot = pd.pivot_table(dfs,
                       index=['openness level'],
                       columns=['policy'],
                       values='final_rewards',
                       aggfunc='mean')

pivot_std = pd.pivot_table(dfs,
                           index=['openness level'],
                           columns=['policy'],
                           values='final_rewards',
                           aggfunc='std')
pivot_std = 1.96 * pivot_std / math.sqrt(45)

pivot.plot(
    kind='bar',
    yerr=pivot_std,
    capsize=5,
    figsize=(10, 6),
)


Concatenating baselines and MOHITO

In [ ]:
mohito_best_policy_df = pd.concat([
    policy_best_checkpoints[openness_level]['mean']
    for openness_level in policy_best_checkpoints
])

print(mohito_best_policy_df)

#merge MOHITOX columns of nonNaNs into one column
mohito_best_policy_df = mohito_best_policy_df.reset_index()
mohito_best_policy_df = mohito_best_policy_df.melt(
    id_vars=['openness level'],
    var_name='policy',
    value_name='final_rewards'
).dropna()

mohito_best_policy_df['policy'] = 'MOHITO'
mohito_best_policy_df.set_index('openness level', inplace=True)

#incorporate ablation results
mohito_original_policy_df = mohito_best_policy_df[mohito_best_policy_df.index > 0]
mohito_ablation_df = mohito_best_policy_df[mohito_best_policy_df.index < 0]

#flip
mohito_ablation_df.index = -mohito_ablation_df.index
mohito_ablation_df['policy'] = mohito_ablation_df['policy'] + ' (Ablation)'
mohito_best_policy_df = pd.concat([mohito_original_policy_df, mohito_ablation_df])


print(mohito_best_policy_df)

# Merge MOHITO and baseline dataframes
pivot['mohito'] = mohito_best_policy_df['final_rewards'][mohito_best_policy_df['policy'] == 'MOHITO']
pivot['mohito (Ablation)'] = mohito_best_policy_df['final_rewards'][mohito_best_policy_df['policy'] == 'MOHITO (Ablation)']
print(pivot)

and ci

In [ ]:
mohito_best_std_df = pd.concat([
    policy_best_checkpoints[openness_level]['ci']
    for openness_level in policy_best_checkpoints
])

print(mohito_best_std_df)

#merge MOHITOX columns of nonNaNs into one column
mohito_best_std_df = mohito_best_std_df.reset_index()
mohito_best_std_df = mohito_best_std_df.melt(
    id_vars=['openness level'],
    var_name='policy',
    value_name='final_rewards'
).dropna()

mohito_best_std_df['policy'] = 'MOHITO'
mohito_best_std_df.set_index('openness level', inplace=True)


#incorporate ablation results
mohito_original_std_df = mohito_best_std_df[mohito_best_std_df.index > 0]
mohito_ablation_df = mohito_best_std_df[mohito_best_std_df.index < 0]

#flip
mohito_ablation_df.index = -mohito_ablation_df.index
mohito_ablation_df['policy'] = mohito_ablation_df['policy'] + ' (Ablation)'
mohito_best_std_df = pd.concat([mohito_original_std_df, mohito_ablation_df])



print(mohito_best_std_df)

# Merge MOHITO and baseline dataframes
pivot_std['mohito'] = mohito_best_std_df['final_rewards'][mohito_best_std_df['policy'] == 'MOHITO']
pivot_std['mohito (Ablation)'] = mohito_best_std_df['final_rewards'][mohito_best_std_df['policy'] == 'MOHITO (Ablation)']
print(pivot_std)

Preparing fire put/burn out times and counts

In [ ]:
#starting a new dfs here
dfs_puts = pd.concat([
    pd.read_csv(os.path.join(root, file))
    for root, _, files in os.walk(baseline_output_folder) for file in files
    if file.endswith('.csv')
], ignore_index=True)

#get groups
dfs_puts['policy'] = dfs_puts['description'].apply(
    lambda x: x.split('_')[0].split(';')[1])
dfs_puts['openness level'] = dfs_puts['description'].apply(
    lambda x: int(x.split('_')[1].split(';')[1]))
dfs_puts['starting state'] = dfs_puts['description'].apply(
    lambda x: int(x.split('_')[2].split(';')[1]))
dfs_puts['episodes'] = dfs_puts['description'].apply(
    lambda x: int(x.split('_')[3].split(';')[1]))
dfs_puts.drop(columns=['description'], inplace=True)

#add MOHITO
mohito_dict = {ol: (path, policy_best_checkpoints[ol]['checkpoint']) for ol, path in policy_eval_outputs.items()}
mohito_dfs_puts = pd.concat([
    policy_best_checkpoints[openness_level]['dfs'][policy_best_checkpoints[openness_level]['dfs']['policy'] == policy_best_checkpoints[openness_level]['checkpoint'].item()].copy()
    for openness_level in mohito_dict
])


#handle the ablation
mohito_ablation_dfs_puts = mohito_dfs_puts['openness level'] < 0
mohito_dfs_puts['policy'] = 'mohito'
mohito_ablation_policy_labels = mohito_dfs_puts[mohito_ablation_dfs_puts]['policy'].apply(lambda x: f'{x} (Ablation)')
mohito_dfs_puts.loc[mohito_ablation_dfs_puts, 'policy'] = mohito_ablation_policy_labels
mohito_dfs_puts['openness level'] = mohito_dfs_puts['openness level'].abs()

#add MOHITO to the dfs
dfs_puts = pd.concat([dfs_puts, mohito_dfs_puts], ignore_index=True)


#get time column as np array
dfs_puts = dfs_puts[['policy', 'step', 'openness level', 'episodes', 'starting state', 'infos/just_put_out_time',
    'infos/just_put_out_ftype', 'infos/just_burned_out_time', 'infos/just_burned_out_ftype']]

grouping = dfs_puts.groupby(['policy', 'openness level', 'starting state', 'episodes'])


grouped_times = {p:[] for p in dfs_puts['policy'].unique()}


# Iterate through policies and openness levels to get the times and types of put out and burned out fires
for (name, group) in grouping:

    group = group.reset_index()

    try:
        group['just_put_out_time'] = group['infos/just_put_out_time'].apply(arrayify)
    except:
        print('hh')

    group['just_burned_out_time'] = group['infos/just_burned_out_time'].apply(arrayify)
    group['just_put_out_ftype'] = group['infos/just_put_out_ftype'].apply(arrayify)
    group['just_burned_out_ftype'] = group['infos/just_burned_out_ftype'].apply(arrayify)

    #create stacked_cols
    put_outs = pd.DataFrame({
        'time': np.concatenate(group['just_put_out_time'].values),
        'fire_type': np.concatenate(group['just_put_out_ftype'].values),
        #repeat for the shape of each embeded array
        'step': np.concatenate([[group['step'].values[i],] * group['just_put_out_time'].loc[i].shape[0] for i in range(len(group))]),
    })
    put_outs['burned_out'] = False

    burn_outs = pd.DataFrame({
        'time': np.concatenate(group['just_burned_out_time'].values),
        'fire_type': np.concatenate(group['just_burned_out_ftype'].values),
        'step': np.concatenate([[group['step'].values[i],] * group['just_burned_out_time'].loc[i].shape[0] for i in range(len(group))]),
    })
    burn_outs['burned_out'] = True

    stacked_cols = pd.concat([put_outs, burn_outs], ignore_index=True)
    stacked_cols['policy'] = name[0]
    stacked_cols['openness_level'] = name[1]
    stacked_cols['starting_state'] = name[2]
    stacked_cols['episodes'] = name[3]
    grouped_times[name[0]].append(stacked_cols)


grouped_times = {
    policy: pd.concat(times, ignore_index=True).reset_index()
    for policy, times in grouped_times.items()
}

## Analysis and Plotting

### Rewards

In [ ]:
print(pivot.columns)

pivot.rename(columns=policy_renaming, inplace=True)
pivot_std.rename(columns=policy_renaming, inplace=True)

#filter to just the renamed columns
pivot = pivot[policy_order]
pivot_std = pivot_std[policy_order]

p1 = pivot.copy()
p1_std = pivot_std.copy()


In [ ]:
p1.plot(kind='bar',
           yerr=p1_std,
           capsize=5,
           figsize=(7, 3),
           title='Average Final Rewards by Openness Level and Policy',
           color=colors,
           width=0.8,)

plt.title('')
plt.yticks(size=12)
plt.xticks(size=12, rotation=0)
plt.ylim(-800, 1400)
plt.xlabel('Openness Level', size=14)
plt.ylabel('Mean Reward (with CI)', size=14)

plt.legend(title=None, fontsize=12, labels=policy_order, ncol=5, prop={'size':10}, loc='lower center', bbox_to_anchor=(0.5, -0.41), columnspacing=1.5, handletextpad=0.5, handlelength=1.5, borderaxespad=0.5)


print(p1)

# plt.text(.73, -500, '-974', fontsize=12, backgroundcolor='white', color='black')
# plt.text(1.18, -500, '-1030', fontsize=12, backgroundcolor='white', color='black')
# plt.text(1.73, -500, '-988', fontsize=12, backgroundcolor='white', color='black')
# plt.text(2.18, -500, '-1277', fontsize=12, backgroundcolor='white', color='black')

plt.savefig('wildfire_rewards.pdf', bbox_inches='tight', dpi=300, transparent=True)

### Wilcoxon

over all openness levels

In [ ]:
#add MOHITO
mohito_dict = {ol: (path, policy_best_checkpoints[ol]['checkpoint']) for ol, path in policy_eval_outputs.items()}

mohito_dfs = pd.concat([
    policy_best_checkpoints[openness_level]['dfs'][policy_best_checkpoints[openness_level]['dfs']['policy'] == policy_best_checkpoints[openness_level]['checkpoint'].item()].copy()
    for openness_level in mohito_dict
])


#handle the ablation
mohito_ablation_dfs = mohito_dfs['openness level'] < 0
mohito_dfs['policy'] = 'mohito'
mohito_ablation_policy_labels = mohito_dfs[mohito_ablation_dfs]['policy'].apply(lambda x: f'{x} (Ablation)')
mohito_dfs.loc[mohito_ablation_dfs, 'policy'] = mohito_ablation_policy_labels
mohito_dfs['openness level'] = mohito_dfs['openness level'].abs()

#add MOHITO to the dfs
df_stats = pd.concat([dfs.copy(), mohito_dfs], ignore_index=True).copy()

#sum over steps
df_stats = df_stats.groupby(['policy', 'openness level', 'episodes','starting state'])[reward_cols].sum().reset_index()
#mean over agent rewards
df_stats['final_rewards'] = df_stats[reward_cols].mean(axis=1)
#select starting states
df_stats = df_stats[df_stats['starting state'].isin(use_these_starting_states)]
#mean over starting states
df_stats = df_stats.groupby(['policy', 'openness level', 'episodes']).mean().reset_index()

#sort values (to ensure alignment for the Wilcoxon test)
df_stats.sort_values(by=['policy','openness level','episodes'], inplace=True)

dfg = df_stats.groupby(['policy'])

for (policy, group) in dfg:

    if policy == ('mohito', ):
        continue

    x = group.reset_index()
    y = dfg.get_group(('mohito', )).reset_index()

    #confirm correct indices
    assert (x['openness level'] == y['openness level']).all(), "Openness levels do not match between groups for Wilcoxon test."
    assert (x['episodes'] == y['episodes']).all(), "Episodes do not match between groups for Wilcoxon test."
    assert (x['starting state'] == y['starting state']).all(), "Starting states do not match between groups for Wilcoxon test."


    print(f'Wilcoxon test: {policy} - MOHITO')
    stat, p_value = stats.wilcoxon(
        x['final_rewards'],
        y['final_rewards'])
    print(f'Statistic: {stat}, p-value: {p_value}')


over individual openness levels

In [ ]:
#add MOHITO
mohito_dict = {
    ol: (path, policy_best_checkpoints[ol]['checkpoint'])
    for ol, path in policy_eval_outputs.items()
}

mohito_dfs = pd.concat([
    policy_best_checkpoints[openness_level]['dfs'][policy_best_checkpoints[openness_level]['dfs']['policy'] == policy_best_checkpoints[openness_level]['checkpoint'].item()].copy()
    for openness_level in mohito_dict
])


#handle the ablation
mohito_ablation_dfs = mohito_dfs['openness level'] < 0
mohito_dfs['policy'] = 'mohito'
mohito_ablation_policy_labels = mohito_dfs[mohito_ablation_dfs]['policy'].apply(lambda x: f'{x} (Ablation)')
mohito_dfs.loc[mohito_ablation_dfs, 'policy'] = mohito_ablation_policy_labels
mohito_dfs['openness level'] = mohito_dfs['openness level'].abs()

# add MOHITO to the dfs
df_stats = pd.concat([dfs.copy(), mohito_dfs], ignore_index=True).copy()

# sum over steps
df_stats = df_stats.groupby(['policy', 'openness level', 'episodes', 'starting state'])[reward_cols].sum().reset_index()
# mean over agent rewards
df_stats['final_rewards'] = df_stats[reward_cols].mean(axis=1)
# select starting states
df_stats = df_stats[df_stats['starting state'].isin(use_these_starting_states)]
# mean over starting states
df_stats = df_stats.groupby(['policy', 'openness level', 'episodes']).mean().reset_index()

#sort values (to ensure alignment for the Wilcoxon test)
df_stats.sort_values(by=['policy','openness level','episodes'], inplace=True)
dfg = df_stats.groupby(['policy', 'openness level'])

for (policy, group) in dfg:

    if policy == ('mohito', policy[1]):
        continue

    print(f'Wilcoxon test: {policy} - MOHITO')
    stat, p_value = stats.wilcoxon(
        group['final_rewards'],
        dfg.get_group(('mohito', policy[1]))['final_rewards'])
    print(f'Statistic: {stat}, p-value: {p_value}')
    print('---------')

### Task (wildfire) duration

In [ ]:
gdf = pd.concat(grouped_times.values(), ignore_index=True)

gdf = gdf[gdf['policy'] != 'NoopBaseline'].copy()
gdf = gdf[['time', 'openness_level', 'policy', 'starting_state','episodes']].groupby(['policy', 'openness_level', 'episodes', 'starting_state'])['time'].mean().reset_index()
gdf = gdf.groupby(['policy', 'openness_level', 'episodes']).mean().reset_index()

pivot = pd.pivot_table(gdf,
    index = 'openness_level',
    columns = 'policy',
    values = 'time',
    aggfunc='mean',
    observed=True #don't consider the possibility of episodes with no fires. Shouldn't be possible...
)

pivot_std = pd.pivot_table(gdf,
    index = 'openness_level',
    columns = 'policy',
    values = 'time',
    aggfunc='std',
    observed=True #don't consider the possibility of episodes with no fires. Shouldn't be possible...
)
pivot_std = 1.96 * pivot_std / math.sqrt(45)

pivot.rename(columns=policy_renaming, inplace=True)
pivot_std.rename(columns=policy_renaming, inplace=True)


pivot = pivot.reindex(columns=policy_order, fill_value=0)
pivot_std = pivot_std.reindex(columns=policy_order, fill_value=0)

pivot.plot(kind='bar',
           yerr=pivot_std,
           capsize=5,
           figsize=(7, 4),
           width=.8,
           title='Average Time to Put Out Fires by Openness Level and Policy',
           color=colors)

plt.title('')
plt.legend(title=None, framealpha=0.0, fontsize=12, labels=policy_order, ncol=3, prop={'size':12}, loc='upper left')
plt.yticks(size=14)
plt.xticks(size=14, rotation=0)
plt.ylim(0, 10)
plt.ylabel("Fire Duration (in timesteps)", size=14)
plt.xlabel("Openness Level",size=16)
plt.tight_layout()
plt.savefig('wildfire_duration.pdf')

### Wildfire put-out / burned-out fire counts

In [ ]:
pre_counted_grouped_times = grouped_times.copy()

fire_name = {
    '1.0': 'Small',
    '2.0': 'Medium',
}

#!Important: ensures that episodes without fires are counted as having 0 put-out or burned-out respectively.
for policy in grouped_times:
    for col in ['openness_level', 'policy', 'starting_state', 'episodes', 'fire_type', 'burned_out']:
        pre_counted_grouped_times[policy][col] = pd.Categorical(pre_counted_grouped_times[policy][col].astype(str))
    
counted_grouped_times = pd.concat([
    g.groupby(['policy','openness_level', 'burned_out', 'episodes', 'starting_state', 'fire_type'], observed=False)['time'].count().reset_index().groupby(
        ['policy','openness_level', 'burned_out', 'episodes', 'fire_type'])['time'].mean().reset_index()
    for policy, g in pre_counted_grouped_times.items()
])

for j, is_burned_out in enumerate(['True', 'False']):

    these_grouped_times = counted_grouped_times[counted_grouped_times['burned_out'] == is_burned_out]
    fig, ax = plt.subplots(1, 2, figsize=(10, 4), sharey=False)

    for size in these_grouped_times['fire_type'].unique():


        relabel = {'mohito': 'MOHITO', 'mohito (Ablation)': 'MOHITO-NoTaskNodes', 'FifoBaseline':'FCFS', 'WeakestBaseline': 'NTF', 'RandomBaseline': 'Random'}
        order = ['MOHITO', 'MOHITO-NoTaskNodes', 'FCFS', 'NTF', 'Random']

        ls_grouped_times = these_grouped_times[these_grouped_times['fire_type']  == size].copy()
        ls_grouped_times['policy'] = ls_grouped_times['policy'].replace(relabel)
        ls_grouped_times = ls_grouped_times[~ls_grouped_times['policy'].isin(['NoopBaseline',]) ]
        
        p = sns.boxplot(
            x='openness_level',
            y='time',
            hue='policy',
            data=ls_grouped_times,
            ax=ax[int(float(size))-1],
            width=0.8,
            palette=colors,
            hue_order = order,
            showfliers=False,
            medianprops={'color': 'black','linewidth':2},
        )

        #time here is actually the count of fires (counted off of the burn/put out time column shape groupe by ftype.)
        group_means = ls_grouped_times.groupby(['openness_level', 'policy'], observed=False)['time'].mean().reset_index()

        width = 0.8

        dic_colors = {
            k : (i - (len(order) -1) /2) * (width / len(order))
            for i, k in enumerate(order)
        }

        offset_map = dic_colors
        for i, row in group_means.iterrows():
            x_pos = list(ls_grouped_times['openness_level'].unique()).index(row['openness_level']) + offset_map[row['policy']]
            ax[int(float(size))-1].scatter(x_pos, row['time'], color='red', edgecolors='black',  marker='D', zorder=10)


        ax[int(float(size))-1].yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{int(round(y))}'))

        ax[int(float(size))-1].get_legend().remove()
        ax[int(float(size))-1].set_xlabel("Openness Level", size=14)
        ax[int(float(size))-1].tick_params(axis='x', labelsize=14, rotation=0)
        ax[int(float(size))-1].tick_params(axis='y', labelsize=14)
        ax[int(float(size))-1].set_title(f"{fire_name[size]} Fires", size=16)

        ax[int(float(size)-1)].set_ylabel(f"{fire_name[size]} fires {'burned out' if is_burned_out == 'True' else 'put out'}", size=16)

    plt.tight_layout()
    
    # Externally constructing legend to avoid the mean markers being included in the legend
    legend = []
    if is_burned_out == 'True':
        for i, l in enumerate(policy_order):
            legend.append(
                Line2D([0], [0], color=colors[i], lw=4, label=l)
            )

    
    ax[0].legend(title=None, framealpha=1.0, fontsize=12, labels= policy_order, 
    ncol=5, loc='lower center', bbox_to_anchor=(1.08, -.32), handles=legend)

plt.savefig(f'wildfire_fires_is_burned_out{is_burned_out == 'True'}.pdf', bbox_inches='tight', pad_inches=0.1)
